In [ ]:
# Notebook referente à implementação inicial sem J2 e simulando apenas a ISS.
# Para rodar esse Notebook, comentar a criação do J2 nas linhas:   
#   22:     #a_j2 = np.array([ax_j2, ay_j2, az_j2])
#   23:     return a_kepler #+ a_j2
#   em: src/orbital_mechanics.py

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Configuração do Path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Importações do Projeto
from src.data_handler import load_tles_smart, get_initial_state_and_time, satellites_to_dataframe
from src.orbital_mechanics import runge_kutta_4
from src.constants import RADIUS_EARTH

print("Ambiente configurado e módulos carregados.")

In [ ]:
# 1. Carrega TLEs (usa o cache se existir)
ts, all_satellites = load_tles_smart(max_days=1.0)

# 2. Converte para DataFrame para buscar fácil
df = satellites_to_dataframe(all_satellites)

# 3. Busca a ISS (ZARYA é o nome oficial do módulo principal nos TLEs)
# Ou busca por 'ISS'
target_name = 'ZARYA' 
iss_row = df[df['name'].str.contains(target_name, case=False)].iloc[0]
iss_sat = iss_row['object']

print(f"Satélite Selecionado: {iss_sat.name}")
print(f"ID NORAD: {iss_sat.model.satnum}")

In [ ]:
# --- Parâmetros da Simulação ---
DURATION_HOURS = 1
DELTA_T = 60.0 # segundos
NUM_STEPS = int(DURATION_HOURS * 3600 / DELTA_T)

# 1. Obter Estado Inicial
t0, state_initial = get_initial_state_and_time(iss_sat, ts)

print(f"Iniciando simulação em: {t0.utc_strftime()}")
print(f"Posição Inicial: {state_initial[0:3]} km")

# 2. Rodar RK4 (Seu Modelo)
# OBS: Se o orbital_mechanics.py estiver com J2, isso roda COM J2.
# Para rodar SEM J2, comente a parte do J2 no arquivo .py antes de rodar aqui.
time_series_rk4, state_history_rk4 = runge_kutta_4(state_initial, DELTA_T, NUM_STEPS)

# 3. Rodar Skyfield (Referência SGP4) para os mesmos tempos
r_skyfield_history = np.zeros((NUM_STEPS + 1, 3))

# Cria array de tempos para o Skyfield
t_skyfield_series = ts.tt_jd(t0.tt + time_series_rk4 / 86400.0)

for i, t_sky in enumerate(t_skyfield_series):
    # Acesso direto .position.km (método robusto que usamos)
    r_skyfield_history[i, :] = iss_sat.at(t_sky).position.km

print("Simulações concluídas.")

In [ ]:
# --- Cálculo do Erro Final ---
final_pos_rk4 = state_history_rk4[-1, 0:3]
final_pos_sgp4 = r_skyfield_history[-1, :]

error_vec = final_pos_rk4 - final_pos_sgp4
error_km = np.linalg.norm(error_vec)

print(f"--- RESULTADOS (Após {DURATION_HOURS}h) ---")
print(f"Erro de Posição: {error_km:.4f} km")

# --- Gráfico 3D ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plota RK4
ax.plot(state_history_rk4[:, 0], state_history_rk4[:, 1], state_history_rk4[:, 2], 
        label='RK4 (Seu Modelo)', color='cyan', linewidth=2)

# Plota Skyfield (Referência)
# Usamos 'dotted' ou markers para diferenciar se estiverem muito perto
ax.plot(r_skyfield_history[:, 0], r_skyfield_history[:, 1], r_skyfield_history[:, 2], 
        label='Skyfield (SGP4)', color='red', linestyle='--', linewidth=1)

# Terra
u = np.linspace(0, 2 * np.pi, 50)
v = np.linspace(0, np.pi, 50)
x_earth = RADIUS_EARTH * np.outer(np.cos(u), np.sin(v))
y_earth = RADIUS_EARTH * np.outer(np.sin(u), np.sin(v))
z_earth = RADIUS_EARTH * np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(x_earth, y_earth, z_earth, color='blue', alpha=0.1)

ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title(f'Validação Unitária: {iss_sat.name}\nErro Final: {error_km:.2f} km')
ax.legend()

# Ajuste de escala para ficar bonito
max_range = np.array([state_history_rk4[:, 0].max()-state_history_rk4[:, 0].min(), 
                      state_history_rk4[:, 1].max()-state_history_rk4[:, 1].min(), 
                      state_history_rk4[:, 2].max()-state_history_rk4[:, 2].min()]).max() / 2.0
mid_x = (state_history_rk4[:, 0].max()+state_history_rk4[:, 0].min()) * 0.5
mid_y = (state_history_rk4[:, 1].max()+state_history_rk4[:, 1].min()) * 0.5
mid_z = (state_history_rk4[:, 2].max()+state_history_rk4[:, 2].min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.show()